In [ ]:
from attr.validators import instance_of
#创建集合单元
from sentence_transformers import SentenceTransformer
from pymilvus import FieldSchema, CollectionSchema, DataType, MilvusClient

client = MilvusClient(uri="http://192.168.200.130:19530")

def create_collection():
    # 定义字段 schemas
    id_field = FieldSchema(name="id", dtype=DataType.INT64, auto_id=True, is_primary=True)
    url_field = FieldSchema(name="url", dtype=DataType.VARCHAR, max_length=65535)
    content_field = FieldSchema(name="content", dtype=DataType.VARCHAR, max_length=65535)
    vector_field = FieldSchema(name="content_vector", dtype=DataType.FLOAT_VECTOR, dim=3584)
    publish_time_field = FieldSchema(name="publish_time", dtype=DataType.INT64)

    # 创建 Collection schema
    schema = CollectionSchema(
        fields=[id_field, url_field, content_field, vector_field, publish_time_field],
        description="CUMT_GPT的Qwen数据集"
    )

    # 创建 Collection
    collection_name = "cumt_gpt_qwen"
    if client.has_collection(collection_name):
        client.drop_collection(collection_name)

    client.create_collection(collection_name=collection_name, schema=schema)

    # 为 content_vector 创建索引
    # index_params = client.prepare_index_params()
    # index_params.add_index(
    #     field_name="content_vector",
    #     index_type="IVF_FLAT",
    #     metric_type="COSINE",
    #     params={"nlist": 128}
    # )
    
    # HNSW
    index_params = client.prepare_index_params()
    index_params.add_index(
        field_name="content_vector",
        index_type="HNSW",
        metric_type="COSINE",
        params={"M": 64, "efConstruction": 250}
    )

    client.create_index(
        collection_name=collection_name,
        index_params=index_params,
        sync=False)


create_collection()
client.list_indexes(collection_name="cumt_gpt_qwen")

In [ ]:
# 插入数据单元
import json
from datetime import datetime
from pymilvus import FieldSchema, CollectionSchema, DataType, MilvusClient
from langchain_community.embeddings import OllamaEmbeddings
from sentence_transformers import SentenceTransformer
from sklearn.preprocessing import normalize
# from sonyflake import SonyFlake

client = MilvusClient(uri="http://192.168.200.130:19530")

def emb_text(text):
    # embedding_model = SentenceTransformer('maidalun1020/bce-embedding-base_v1')
    # embedding_vectors = embedding_model.encode(text, batch_size=1024)
    if isinstance(text, str):
        text = [text]
    
    embedding_model = OllamaEmbeddings(model="qwen2.5:7b")
    embedding_vectors = []
    
    for text_i in text:
        embedding_vectors.append(embedding_model.embed_query(text_i))

    # embedding_vectors = normalize(embedding_vectors, norm='l2')
    return embedding_vectors

# 文档列表
# docs = [
#     "Artificial intelligence was founded as an academic discipline in 1956.",
#     "Alan Turing was the first person to conduct substantial research in AI.",
#     "Born in Maida Vale, London, Turing was raised in southern England.",
# ]
# doc = "矿小助是一款由FlyingStudio（中国矿业大学翔工作室）开发维护的校园软件。提供课表、考试、成绩、校车校历、图书馆藏、校卡余额、宿舍电量等查询功能；同时具有课表导入日历、加权成绩计算、校园网自动登录、个性化主题背景等实用功能。"

contents = []

# 打开 JSON 文件
filename = '../raw_data/articles.json'
# filename = ('../raw_data/kxz.json')
# filename = '../raw_data/news.json'
# filename = '../raw_data/矿大新闻网.json'
with open(filename, 'r', encoding='utf-8') as f:
    python_data = json.load(f)
    for data in python_data['data']:
        contents.append(data.get("content"))

vectors = emb_text(contents)

# 要插入的数据
datas = []
i = 0
for data in python_data['data']:
    # 创建一个 SonyFlake 实例
    # flake = SonyFlake()

    # 生成唯一ID
    # sonyflake_id = flake.next_id()
    
    # 使用 strptime 将字符串解析为日期对象
    date_obj = datetime.strptime(data.get("date"), "%Y-%m-%d")
    
    # 使用 strftime 格式化为所需的格式，并将其转换为整数
    date_int = int(date_obj.strftime("%Y%m%d"))

    datas.append({
            # "id": sonyflake_id,
            "url": data.get("url"),
            "content": data.get("content"),
            "content_vector": vectors[i],
            "publish_time": date_int
        })
    i += 1

# 打印每个向量的维度和生成的嵌入
print("向量数量: ", len(vectors))
print("向量维度: ", len(vectors[0]))
# print(contents)
# print("生成的向量:", vectors)

# 插入数据
res = client.insert(collection_name="cumt_gpt_qwen", data=datas)
print(res)


In [ ]:
# 测试向量搜索单元
from langchain_community.embeddings import OllamaEmbeddings
from sentence_transformers import SentenceTransformer
from pymilvus import FieldSchema, CollectionSchema, DataType, MilvusClient
from sklearn.preprocessing import normalize

client = MilvusClient(uri="http://192.168.200.130:19530")

def emb_text(text):
    # embedding_model = SentenceTransformer('maidalun1020/bce-embedding-base_v1')
    # embedding_vectors = embedding_model.encode(text, batch_size=1024)
    if isinstance(text, str):
        text = [text]
    
    embedding_model = OllamaEmbeddings(model="qwen2.5:7b")
    embedding_vectors = []
    
    for text_i in text:
        embedding_vectors.append(embedding_model.embed_query(text_i))
    
    # embedding_vectors = normalize(embedding_vectors, norm='l2')
    return embedding_vectors

doc = "孙杨什么时候来学校"

doc_vector = emb_text(doc)

search_params = {
    "metric_type": "COSINE",
    "params": {}
}

# IVF_FLAT
# res = client.search(
#     collection_name="cumt_gpt_qwen",
#     data=doc_vector,
#     limit=10,
#     output_fields=["id", "url", "content", "publish_time"],
#     search_params=search_params
# )

# HNSW
res = client.search(
  collection_name="cumt_gpt_qwen", # Collection name
  data=doc_vector, # Replace with your query vector
  search_params={
    "metric_type": "COSINE",
    "params": {"ef":150, "radius":0}, # Search parameters
  }, # Search parameters
  limit=10, # Max. number of search results to return
  output_fields=["id","url","content","publish_time"], # Fields to return in the search results
  consistency_level="Bounded"
)


print(res)